# 학습

In [ ]:
!pip install -U transformers==4.43.3 accelerate==0.33.0 peft==0.11.1 trl==0.9.6 datasets==2.20.0 pandas==2.2.2
!pip install -U bitsandbytes==0.43.1
!pip install -U triton==2.3.0


In [3]:
!sudo umount -l /content/drive
!rm -rf /content/drive


umount: /content/drive: not mounted.


In [ ]:
# -*- coding: utf-8 -*-
"""
KD + QLoRA SFT (ChatML) for: text,label,analysis -> {"is_phishing": bool, "reason": str}
- Teacher: Qwen2.5-3B-Instruct (지식 증류용 soft target 제공, 학습 X)
- Student: Qwen2.5-1.5B-Instruct (4bit + QLoRA 학습)
- assistant 구간만 Loss (Hard CE + Soft KD)
- ChatML 안전 생성 / response_template 자동 추출
- 기존 설정(4bit, NEFTune, FlashAttention2 등) 유지
"""

import os, random, json, re
from typing import Dict, Any
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

from datasets import Dataset
from sklearn.model_selection import train_test_split

from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer
)
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import DataCollatorForCompletionOnlyLM
from google.colab import drive
drive.mount('/content/drive')

# ===== 사용자 수정 =====
CSV_PATH    = "/content/drive/MyDrive/KDH/dataset/llm_datasets_0817_flow_utf8.csv"  # text,label,analysis
OUTPUT_DIR  = "/content/drive/MyDrive/KDH/llmlightning/lora_kd_qwen1p5b_from_3b"
TEACHER_MODEL = "Qwen/Qwen2.5-3B-Instruct"
STUDENT_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"

EPOCHS        = 2
BATCH_SIZE    = 1
GRAD_ACCUM    = 16
LR            = 1e-4
WARMUP_RATIO  = 0.10
WEIGHT_DECAY  = 0.10
MAX_GRAD_NORM = 0.30
MAX_LEN       = 1024
VAL_RATIO     = 0.1
SEED          = 42

# KD 하이퍼파라미터
KD_ALPHA       = 0.5      # Hard CE 가중치 (나머지 1-KD_ALPHA가 Soft KD 가중치)
KD_TEMPERATURE = 2.0

# LoRA 설정 (Qwen 계열)
LORA_R         = 32
LORA_ALPHA     = 64
LORA_DROPOUT   = 0.10
TARGET_MODULES = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]

# 기타
NEFTUNE_NOISE_ALPHA = 5           # SFTTrainer 기능이지만, 여기선 참고만 (KD 커스텀이라 미사용)
TEACHER_4BIT        = False       # 메모리 부족하면 True로
# ======================


def set_seed(s: int):
    random.seed(s); np.random.seed(s); torch.manual_seed(s)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(s)


def bf16_supported() -> bool:
    try:
        return torch.cuda.is_available() and torch.cuda.is_bf16_supported()
    except Exception:
        return False


def detect_flash_attn2() -> bool:
    try:
        import flash_attn  # noqa
        return True
    except Exception:
        return False


def get_tokenizer(model_name: str):
    try:
        tok = AutoTokenizer.from_pretrained(model_name, use_fast=True, trust_remote_code=True)
    except Exception:
        tok = AutoTokenizer.from_pretrained(model_name, use_fast=False, trust_remote_code=True)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    return tok


def build_user_content(text: str) -> str:
    """학습 user 메시지 (분석 지시 포함, JSON 외 금지)"""
    lines = [
        "다음 대화를 보고 순수 JSON으로만 답해라.",
        '스키마: {"is_phishing": true|false, "reason": "한국어 1~2문장"}',
        "- JSON 이외의 텍스트 금지.",
        "",
        "[대화]",
        str(text).strip()
    ]
    return "\n".join(lines)


def apply_chat_template_text(tokenizer, system: str, user_content: str, assistant_json: Dict[str, Any]) -> str:
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": json.dumps(assistant_json, ensure_ascii=False)},
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False)


def get_response_template_from_tokenizer(tokenizer) -> str:
    tmpl = tokenizer.apply_chat_template(
        [{"role": "assistant", "content": ""}], tokenize=False, add_generation_prompt=False
    )
    if "<|im_end|>" in tmpl:
        tmpl = tmpl.split("<|im_end|>")[0]
    return tmpl


def main():
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    set_seed(SEED)

    # ============= 1) 데이터 로드/정리 (text,label,analysis) =============
    df = pd.read_csv(CSV_PATH)
    assert all(c in df.columns for c in ["text", "label", "analysis"]), "CSV에 text,label,analysis 컬럼이 필요합니다."

    # text / analysis 정리
    df["text"] = df["text"].astype(str).str.strip()
    df["analysis"] = df["analysis"].astype(str).str.strip()

    # label 표준화
    label_map = {"true":1,"false":0,"t":1,"f":0,"yes":1,"no":0,"y":1,"n":0,"보이스피싱":1,"정상":0}
    def norm_label(x):
        if pd.isna(x): return np.nan
        s = str(x).strip().lower()
        if s in ("", "nan", "none"): return np.nan
        if s in label_map: return label_map[s]
        try:
            v = float(s)
            return 1 if np.isfinite(v) and v >= 0.5 else 0
        except:
            return np.nan

    df["label"] = df["label"].apply(norm_label)
    df["label"] = df["label"].replace([np.inf, -np.inf], np.nan)

    # reason 길이 간단 제한(너무 길면 2문장 수준으로 잘라주기)
    def re_split_sent(s: str):
        # 마침표/느낌표/물음표 OR 닫는 괄호 뒤 공백에서 분리
        return re.split(r'(?<=[.!?])\s+|(?<=\))\s+', s)

    def trim_reason(s: str) -> str:
        s = s.replace("\n", " ").strip()
        parts = [p.strip() for p in re_split_sent(s)]
        return " ".join(parts[:2]) if parts else s

    df["analysis"] = df["analysis"].apply(trim_reason)

    before = len(df)
    df = df.dropna(subset=["text","label","analysis"]).copy()
    df["label"] = df["label"].astype(int)
    after = len(df)
    print(f"[CLEAN] Dropped {before - after} rows with bad fields")

    # 중복 제거
    df = df.drop_duplicates(subset=["text","analysis"]).reset_index(drop=True)

    train_df, val_df = train_test_split(
        df, test_size=VAL_RATIO, random_state=SEED, stratify=df["label"]
    )

    # ============= 2) ChatML 샘플 생성 (reason = analysis 사용) =============
    SYSTEM = (
        "너는 보이스피싱 대화를 감별하는 분석가다. "
        "입력 대화를 보고 **순수 JSON만** 출력한다. "
        '스키마: {"is_phishing": true|false, "reason": "한국어 1~2문장"} '
        "JSON 이외의 텍스트는 절대 출력하지 마라."
    )

    # tokenizer (공유)
    tokenizer = get_tokenizer(STUDENT_MODEL)  # Student 기준으로 패딩/토크나이즈(동일 ChatML)

    def row_to_text(row) -> Dict[str, Any]:
        user_content = build_user_content(row["text"])
        target = {
            "is_phishing": bool(int(row["label"]) == 1),
            "reason": row["analysis"][:400]  # 혹시 모를 과도한 길이 방지
        }
        chat_text = apply_chat_template_text(tokenizer, SYSTEM, user_content, target)
        return {"text": chat_text}

    train_records = [row_to_text(r) for _, r in train_df.iterrows()]
    val_records   = [row_to_text(r) for _, r in val_df.iterrows()]

    train_ds = Dataset.from_list(train_records)
    val_ds   = Dataset.from_list(val_records)

    # ============= 3) Collator (assistant 구간만 Loss 마스킹) =============
    ASSISTANT_PREFIX = get_response_template_from_tokenizer(tokenizer)
    collator = DataCollatorForCompletionOnlyLM(
        response_template=ASSISTANT_PREFIX, tokenizer=tokenizer, mlm=False
    )

    # ============= 4) 모델 로드 =============
    # ---- Student: 1.5B + QLoRA(4bit) ----
    bnb_config_student = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16 if bf16_supported() else torch.float16,
    )
    attn_impl = "flash_attention_2" if detect_flash_attn2() else "eager"

    student = AutoModelForCausalLM.from_pretrained(
        STUDENT_MODEL,
        quantization_config=bnb_config_student,
        device_map="auto",
        trust_remote_code=True,
        attn_implementation=attn_impl,
    )
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    student.gradient_checkpointing_enable()
    student.config.use_cache = False
    student = prepare_model_for_kbit_training(student)

    lora_cfg = LoraConfig(
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        lora_dropout=LORA_DROPOUT,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=TARGET_MODULES,
    )
    student = get_peft_model(student, lora_cfg)
    student.print_trainable_parameters()

    # ---- Teacher: 3B (KD용, 학습 X) ----
    if TEACHER_4BIT:
        bnb_config_teacher = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16 if bf16_supported() else torch.float16,
        )
        teacher = AutoModelForCausalLM.from_pretrained(
            TEACHER_MODEL,
            quantization_config=bnb_config_teacher,
            device_map="auto",
            trust_remote_code=True,
            attn_implementation="eager",  # 안정성 위해 eager 권장
        )
    else:
        teacher = AutoModelForCausalLM.from_pretrained(
            TEACHER_MODEL,
            device_map="auto",
            trust_remote_code=True,
            torch_dtype=torch.bfloat16 if bf16_supported() else torch.float16,
            attn_implementation="eager",
        )
    teacher.eval()
    for p in teacher.parameters():
        p.requires_grad = False

    # ============= 5) KD 전용 Trainer =============
    class KDSFTTrainer(Trainer):
        """
        - inputs: DataCollatorForCompletionOnlyLM가 만든 dict (input_ids, attention_mask, labels)
        - Hard CE: labels != -100 (assistant 구간)
        - Soft KD: teacher/student 분포 KLDiv on assistant 구간 (temperature scaling)
        """
        def __init__(self, *args, teacher_model=None, kd_alpha=0.5, kd_temperature=2.0, **kwargs):
            super().__init__(*args, **kwargs)
            self.teacher = teacher_model.eval()
            self.kd_alpha = kd_alpha
            self.kd_temperature = kd_temperature
            self.ce_loss = torch.nn.CrossEntropyLoss(ignore_index=-100)

        @torch.no_grad()
        def _teacher_logits(self, inputs):
            # teacher 디바이스로 이동
            tdev = next(self.teacher.parameters()).device
            tinp = {k: v.to(tdev) for k, v in inputs.items() if isinstance(v, torch.Tensor)}
            out = self.teacher(**tinp)
            return out.logits

        def compute_loss(self, model, inputs, return_outputs=False):
            labels = inputs.get("labels")
            outputs_s = model(**inputs)
            logits_s = outputs_s.logits  # [B, T, V]

            # Hard CE (assistant 구간)
            loss_ce = self.ce_loss(
                logits_s.view(-1, logits_s.size(-1)),
                labels.view(-1)
            )

            # Soft KD (teacher 분포)
            with torch.no_grad():
                logits_t = self._teacher_logits(inputs)  # [B, T, V]

            # assistant 구간 마스크
            mask = labels.ne(-100)  # [B, T]
            if mask.any():
                T = self.kd_temperature
                # 마스크된 위치만 추출 → [N, V]
                s = (logits_s / T)[mask]
                t = (logits_t / T)[mask]
                loss_kd = F.kl_div(
                    F.log_softmax(s, dim=-1),
                    F.softmax(t, dim=-1),
                    reduction="batchmean"
                ) * (T * T)
            else:
                loss_kd = torch.zeros((), device=logits_s.device)

            loss = self.kd_alpha * loss_ce + (1.0 - self.kd_alpha) * loss_kd
            return (loss, outputs_s) if return_outputs else loss

    # ============= 6) 학습 설정 =============
    use_bf16 = bf16_supported()
    train_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=GRAD_ACCUM,
        learning_rate=LR,
        lr_scheduler_type="cosine",
        warmup_ratio=WARMUP_RATIO,
        weight_decay=WEIGHT_DECAY,
        max_grad_norm=MAX_GRAD_NORM,
        logging_steps=20,
        save_steps=500,
        evaluation_strategy="steps",
        eval_steps=500,
        save_total_limit=2,
        bf16=use_bf16,
        fp16=(not use_bf16),
        gradient_checkpointing=True,
        optim="paged_adamw_8bit",
        group_by_length=True,
        dataloader_pin_memory=True,
        report_to="none",
        ddp_find_unused_parameters=False if torch.cuda.device_count() > 1 else None,
        max_steps=-1,
        save_safetensors=True,
    )

    # ============= 7) Trainer 생성/학습/저장 =============
    trainer = KDSFTTrainer(
        model=student,
        tokenizer=tokenizer,
        args=train_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        data_collator=collator,
        teacher_model=teacher,
        kd_alpha=KD_ALPHA,
        kd_temperature=KD_TEMPERATURE,
    )

    trainer.train()
    trainer.model.save_pretrained(OUTPUT_DIR)
    tokenizer.save_pretrained(OUTPUT_DIR)
    print(f"[OK] KD + QLoRA LoRA saved to: {OUTPUT_DIR}")


if __name__ == "__main__":
    main()


Mounted at /content/drive
[CLEAN] Dropped 0 rows with bad fields


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

AttributeError: /usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cquantize_blockwise_fp16_nf4

# 추론

In [ ]:
# -*- coding: utf-8 -*-
import json, re, torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# ========= 모델 경로 =========
BASE = "Qwen/Qwen2.5-3B-Instruct"
LORA = "/content/drive/MyDrive/KDH/LLM/lora_label_analysis"  # 네가 방금 학습한 LoRA 경로

# ========= 로드 =========
tok = AutoTokenizer.from_pretrained(BASE, use_fast=True, trust_remote_code=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

base = AutoModelForCausalLM.from_pretrained(
    BASE, device_map="auto", torch_dtype="auto", trust_remote_code=True
).eval()
model = PeftModel.from_pretrained(base, LORA).eval()

# ========= 프롬프트 (학습 스키마와 정확히 일치) =========
SYSTEM = (
    "너는 보이스피싱 탐지 전문가다. 반드시 순수 JSON 한 덩어리만 출력한다. "
    '스키마는 {"is_phishing": true|false, "reason": "한국어 1~2문장"} 이다. '
    "JSON 밖의 텍스트/주석/라벨/마크다운은 금지한다."
)

USER_TMPL = (
    "다음 대화를 읽고 실제 공공기관 정상 절차와 비교하여 왜 다른지 요약하고, "
    "수법(기관 사칭/안전계좌 유도/환급 미끼/원격앱 유도 등)을 근거로 보이스피싱 여부를 판단하라. "
    '오직 {"is_phishing": true|false, "reason": "한국어 1~2문장"} 형태의 JSON만 출력하라.\n'
    "대화:\n\"\"\"\n{TEXT}\n\"\"\""
)

# ========= JSON 회수(견고) =========
def _extract_balanced_json(text: str):
    in_str = False; esc = False; depth = 0; start = -1
    for i, ch in enumerate(text):
        if ch == '"' and not esc: in_str = not in_str
        esc = (ch == '\\') and not esc
        if in_str: continue
        if ch == '{':
            if depth == 0: start = i
            depth += 1
        elif ch == '}':
            if depth > 0:
                depth -= 1
                if depth == 0 and start != -1:
                    block = text[start:i+1]
                    try:
                        return json.loads(block)
                    except Exception:
                        pass
    raise ValueError("No valid JSON object found")

_SENT_SPLIT = re.compile(r'(?<=[.!?])\s+|(?<=\))\s+')

def _postprocess(js: dict) -> dict:
    out = {}
    out["is_phishing"] = bool(js.get("is_phishing", False))
    reason = str(js.get("reason", "")).strip()
    if reason:
        parts = [p.strip() for p in _SENT_SPLIT.split(reason) if p.strip()]
        reason = " ".join(parts[:2])  # 최대 2문장
    out["reason"] = reason
    return out

# ========= 추론 =========
def infer(text: str, max_new_tokens: int = 160, deterministic: bool = True) -> dict:
    user = USER_TMPL.replace("{TEXT}", text)
    messages = [
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": user},
    ]

    prompt = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tok(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        out_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=not deterministic,   # 기본은 일관성 우선
            top_p=0.9,
            temperature=0.6,
            repetition_penalty=1.08,
            no_repeat_ngram_size=4,
            eos_token_id=tok.eos_token_id,
        )

    gen = tok.decode(out_ids[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()
    try:
        js = _extract_balanced_json(gen)
    except Exception:
        return {"error": gen}

    # 스키마 고정 + 2문장 컷
    js = {k: js[k] for k in js.keys() if k in ("is_phishing", "reason")}
    return _postprocess(js)

# ========= 사용 예시 =========
if __name__ == "__main__":
    text = (
        "신용카드와 대포통장을 압수하는 과정에서 귀하의 명의로 된 농협과 하나은행 통장 두 개가 발견되어 연락드렸습니다. 이 통장들에 대해 알고 계신 사실이 있으신가요?"
    )
    print(infer(text))
